In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the datasets
train_data = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "text"])
test_data = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "text"])

In [ ]:
# Explore the loaded datasets
print("Train data shape:", train_data.shape)
print("Valid data shape:", test_data.shape)
print(train_data.head())
print(test_data.head())

In [ ]:
train_data['label']=train_data['label'].map({"ham":1.0,'spam':0.0})
test_data['label']=test_data['label'].map({"ham":1.0,'spam':0.0})
train_data['label'] = np.asarray(train_data['label']).astype(np.float32)
test_data['label'] = np.asarray(test_data['label']).astype(np.float32)


In [ ]:
# create dataset
train_data = tf.data.Dataset.from_tensor_slices((train_data['text'], train_data['label']))
test_data = tf.data.Dataset.from_tensor_slices((test_data['text'], test_data['label']))


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 1000

train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(train_data.map(lambda text, label: text))
vocab = np.array(encoder.get_vocabulary())

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

assert [layer.supports_masking for layer in model.layers] == [False, True, True, True, True]

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_data, epochs=10, validation_data=test_data, validation_steps=30)

test_loss, test_acc = model.evaluate(test_data)

# save model
model.save('model')

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    predictions = model.predict(np.array([pred_text]))[0][0]

    if round(predictions) >= 1:
        return [predictions, "ham"]
    return [predictions, "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
